In [1]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm.notebook import tqdm
import pickle

from plot_3d_helpers import expanded_detector_boundary_points, detector_boundary_points
from plot_3d_helpers import fps_clustering_downsample, get_min_dists, energy_weighted_density_sampling


# Loading File

In [13]:
f = uproot.open("input_files/bdt_convert_superunified_bnb_ncpi0_full_spacepoints.root")


In [16]:
# no information about parentage in the EDeps
# see https://github.com/uboone/lardataobj/blob/338c038932129d944b0a693e636329941fd6ec45/lardataobj/Simulation/SimEnergyDeposit.h

for item in f["wcpselection"]["T_spacepoints"].items():
    if "TrueEDep_spacepoints" in str(item):
        print(item)


('TrueEDep_spacepoints_startx', <TBranchElement 'TrueEDep_spacepoints_startx' at 0x0006e42b1b70>)
('TrueEDep_spacepoints_starty', <TBranchElement 'TrueEDep_spacepoints_starty' at 0x0006e42b24d0>)
('TrueEDep_spacepoints_startz', <TBranchElement 'TrueEDep_spacepoints_startz' at 0x0006e42b2e30>)
('TrueEDep_spacepoints_endx', <TBranchElement 'TrueEDep_spacepoints_endx' at 0x0006e42b3790>)
('TrueEDep_spacepoints_endy', <TBranchElement 'TrueEDep_spacepoints_endy' at 0x0006e42d9150>)
('TrueEDep_spacepoints_endz', <TBranchElement 'TrueEDep_spacepoints_endz' at 0x0006e42d86a0>)
('TrueEDep_spacepoints_edep', <TBranchElement 'TrueEDep_spacepoints_edep' at 0x00030fb2d1b0>)
('TrueEDep_spacepoints_pdg', <TBranchElement 'TrueEDep_spacepoints_pdg' at 0x00030fb2db10>)


In [45]:
print("looking at the geant truth information for a shower in WC")

wc_geant_dic = f["wcpselection"]["T_PFeval"].arrays(["truth_id", "truth_mother", "truth_pdg", "truth_startMomentum"], library="np")

one_event_wc_geant_dic = {}

for k, v in wc_geant_dic.items():
    one_event_wc_geant_dic[k] = v[61]

# finding the primary pi0
primary_pi0_id = -1
for i in range(len(one_event_wc_geant_dic["truth_id"])):
    if one_event_wc_geant_dic["truth_pdg"][i] == 111 and one_event_wc_geant_dic["truth_mother"][i] == 0:
        primary_pi0_id = one_event_wc_geant_dic["truth_id"][i]
        print("found the primary pi0")
        break

# finding the daughters of the primary pi0
daughters_of_primary_pi0_ids = []
for i in range(len(one_event_wc_geant_dic["truth_id"])):
    if one_event_wc_geant_dic["truth_mother"][i] == primary_pi0_id:
        print("  daughter of primary pi0")
        print("    truth_id:", one_event_wc_geant_dic["truth_id"][i])
        print("    truth_pdg:", one_event_wc_geant_dic["truth_pdg"][i])
        print("    truth_startMomentum:", one_event_wc_geant_dic["truth_startMomentum"][i])
        daughters_of_primary_pi0_ids.append(one_event_wc_geant_dic["truth_id"][i])

# finding all descendants of the first daughter of the primary pi0
first_daughter_and_descendants_ids = [daughters_of_primary_pi0_ids[0]]
first_daughter_descendants_pdgs = []
num_added = 1
while num_added != 0:
    #print("looping over particles")
    num_added = 0
    for i in range(len(one_event_wc_geant_dic["truth_id"])):
        if one_event_wc_geant_dic["truth_mother"][i] in first_daughter_and_descendants_ids and one_event_wc_geant_dic["truth_id"][i] not in first_daughter_and_descendants_ids:
            first_daughter_and_descendants_ids.append(one_event_wc_geant_dic["truth_id"][i])
            first_daughter_descendants_pdgs.append(one_event_wc_geant_dic["truth_pdg"][i])
            num_added += 1

# finding the descendants of the second daughter of the primary pi0
second_daughter_and_descendants_ids = [daughters_of_primary_pi0_ids[1]]
second_daughter_descendants_pdgs = []
num_added = 1
while num_added != 0:
    #print("looping over particles")
    num_added = 0
    for i in range(len(one_event_wc_geant_dic["truth_id"])):
        if one_event_wc_geant_dic["truth_mother"][i] in second_daughter_and_descendants_ids and one_event_wc_geant_dic["truth_id"][i] not in second_daughter_and_descendants_ids:
            second_daughter_and_descendants_ids.append(one_event_wc_geant_dic["truth_id"][i])
            second_daughter_descendants_pdgs.append(one_event_wc_geant_dic["truth_pdg"][i])
            num_added += 1

print("first daughter descendants pdgs: ", first_daughter_descendants_pdgs)
print("second daughter descendants pdgs: ", second_daughter_descendants_pdgs)


looking at the geant truth information for a shower in WC
found the primary pi0
  daughter of primary pi0
    truth_id: 21
    truth_pdg: 22
    truth_startMomentum: [-0.10130839 -0.13959165  0.3532334   0.3930942 ]
  daughter of primary pi0
    truth_id: 22
    truth_pdg: 22
    truth_startMomentum: [-0.06241408  0.0257559   0.16856109  0.18158118]
first daughter descendants pdgs:  [11, -11, 22, 22, 22, 22, 22, 11, 22, 11, 11, -11, 11, -11, 22, 11, 11, -11]
second daughter descendants pdgs:  [11, -11, 22, 22, 22, 22, 22, -11, -11, 11]


In [48]:
print("looking at the geant truth information for a shower in Pandora")

f["nuselection"]["NeutrinoSelectionFilter"].items()

looking at the geant truth information for a shower in Pandora


[('selected', <TBranch 'selected' at 0x0006f80a4430>),
 ('run', <TBranch 'run' at 0x0006f80a6560>),
 ('sub', <TBranch 'sub' at 0x0006f80a5000>),
 ('evt', <TBranch 'evt' at 0x0006f80c2d40>),
 ('trk_id', <TBranch 'trk_id' at 0x0006f80c0fd0>),
 ('shr_id', <TBranch 'shr_id' at 0x0006f808f2e0>),
 ('trk2_id', <TBranch 'trk2_id' at 0x0006f808c550>),
 ('shr2_id', <TBranch 'shr2_id' at 0x0006f808ce20>),
 ('trk3_id', <TBranch 'trk3_id' at 0x0006f808dd20>),
 ('shr3_id', <TBranch 'shr3_id' at 0x0006f808f040>),
 ('shr_energy_tot', <TBranch 'shr_energy_tot' at 0x0006f808e170>),
 ('shr_energy', <TBranch 'shr_energy' at 0x0006f0cbb4c0>),
 ('shr_energy_second', <TBranch 'shr_energy_second' at 0x0006f7ffcdf0>),
 ('shr_energy_third', <TBranch 'shr_energy_third' at 0x0006f7ffc0d0>),
 ('shr_energy_tot_cali', <TBranch 'shr_energy_tot_cali' at 0x0006f7fff3d0>),
 ('shr_energy_cali', <TBranch 'shr_energy_cali' at 0x0006f7ffc0a0>),
 ('shr_energy_second_cali',
  <TBranch 'shr_energy_second_cali' at 0x0006f7ffcb5

In [51]:
f["nuselection"]["NeutrinoSelectionFilter"]["pi0truth_gamma1_edep"].arrays(library="np")

{'pi0truth_gamma1_edep': array([192.214  , 260.66714, 272.84106, ..., 290.34164, 139.21387,
        134.24088], dtype=float32)}

In [3]:
rse = f["wcpselection"]["T_eval"].arrays(["run", "subrun", "event"], library="np")

true_nu_vtx = f["wcpselection"]["T_eval"].arrays(["truth_vtxX", "truth_vtxY", "truth_vtxZ"], library="np")
true_nu_vtx = np.stack([true_nu_vtx["truth_vtxX"], true_nu_vtx["truth_vtxY"], true_nu_vtx["truth_vtxZ"]], axis=-1)

reco_nu_vtx = f["wcpselection"]["T_PFeval"].arrays(["reco_nuvtxX", "reco_nuvtxY", "reco_nuvtxZ"], library="np")
reco_nu_vtx = np.stack([reco_nu_vtx["reco_nuvtxX"], reco_nu_vtx["reco_nuvtxY"], reco_nu_vtx["reco_nuvtxZ"]], axis=-1)


In [4]:
num_events = 70
# num_events = len(truth_vtx["truth_vtxX"])

spacepoints = f["wcpselection"]["T_spacepoints"].arrays(["Tcluster_spacepoints_x", 
                                                                 "Tcluster_spacepoints_y", 
                                                                 "Tcluster_spacepoints_z", 
                                                                 #"Tcluster_spacepoints_q",
                                                                 "Trec_spacepoints_x", 
                                                                 "Trec_spacepoints_y", 
                                                                 "Trec_spacepoints_z", 
                                                                 #"Trec_spacepoints_q",
                                                                 "TrueEDep_spacepoints_startx",
                                                                 "TrueEDep_spacepoints_starty",
                                                                 "TrueEDep_spacepoints_startz",
                                                                 "TrueEDep_spacepoints_endx",
                                                                 "TrueEDep_spacepoints_endy",
                                                                 "TrueEDep_spacepoints_endz",
                                                                 "TrueEDep_spacepoints_edep",
                                                                 ], 
                                                                 entry_start=0, entry_stop=num_events, library="np")

Tcluster_spacepoints = []
Trec_spacepoints = []
TrueEDep_spacepoints = []
TrueEDep_spacepoints_edep = []
for event_i in range(num_events):
    Tcluster_spacepoints.append(np.stack([spacepoints["Tcluster_spacepoints_x"][event_i],
                                         spacepoints["Tcluster_spacepoints_y"][event_i],
                                         spacepoints["Tcluster_spacepoints_z"][event_i]], axis=-1))
    Trec_spacepoints.append(np.stack([spacepoints["Trec_spacepoints_x"][event_i],
                                      spacepoints["Trec_spacepoints_y"][event_i],
                                      spacepoints["Trec_spacepoints_z"][event_i]], axis=-1))
    starts = np.stack([spacepoints["TrueEDep_spacepoints_startx"][event_i],
                       spacepoints["TrueEDep_spacepoints_starty"][event_i],
                       spacepoints["TrueEDep_spacepoints_startz"][event_i]], axis=-1)
    midpoints = np.stack([(spacepoints["TrueEDep_spacepoints_startx"][event_i] + spacepoints["TrueEDep_spacepoints_endx"][event_i])/2,
                          (spacepoints["TrueEDep_spacepoints_starty"][event_i] + spacepoints["TrueEDep_spacepoints_endy"][event_i])/2,
                          (spacepoints["TrueEDep_spacepoints_startz"][event_i] + spacepoints["TrueEDep_spacepoints_endz"][event_i])/2], axis=-1)
    ends = np.stack([spacepoints["TrueEDep_spacepoints_endx"][event_i],
                     spacepoints["TrueEDep_spacepoints_endy"][event_i],
                     spacepoints["TrueEDep_spacepoints_endz"][event_i]], axis=-1)
    TrueEDep_spacepoints.append(np.concatenate([starts, midpoints, ends], axis=0))

    # assuming a third of the energy at the start, midpoint, and end
    TrueEDep_spacepoints_edep.append(np.concatenate([spacepoints["TrueEDep_spacepoints_edep"][event_i]/3,
                                                    spacepoints["TrueEDep_spacepoints_edep"][event_i]/3,
                                                    spacepoints["TrueEDep_spacepoints_edep"][event_i]/3], axis=0))

del spacepoints
del f


# Downsample Spacepoints

In [5]:
close_to_reco_nu_vtx_threshold = 200
recalculate_downsampling = True

if recalculate_downsampling:
    downsampled_Tcluster_spacepoints = {}
    downsampled_Trec_spacepoints = {}
    downsampled_TrueEDep_spacepoints = {}
    for event_i in tqdm(range(num_events)):

        nearby_reco_nu_vtx_indices = np.where(np.sqrt((Tcluster_spacepoints[event_i][:, 0] - reco_nu_vtx[event_i][0])**2
                                                    + (Tcluster_spacepoints[event_i][:, 1] - reco_nu_vtx[event_i][1])**2
                                                    + (Tcluster_spacepoints[event_i][:, 2] - reco_nu_vtx[event_i][2])**2) < close_to_reco_nu_vtx_threshold)[0]
        Tcluster_spacepoints_near_reco_nu_vtx = Tcluster_spacepoints[event_i][nearby_reco_nu_vtx_indices, :]
        downsampled_Tcluster_spacepoints[event_i] = fps_clustering_downsample(Tcluster_spacepoints_near_reco_nu_vtx, 500)

        downsampled_Trec_spacepoints[event_i] = fps_clustering_downsample(Trec_spacepoints[event_i], 200)

        downsampled_TrueEDep_spacepoints[event_i] = energy_weighted_density_sampling(TrueEDep_spacepoints[event_i], TrueEDep_spacepoints_edep[event_i], 500)
        

    with open("downsampled_spacepoints.pkl", "wb") as f:
        pickle.dump((downsampled_Tcluster_spacepoints, downsampled_Trec_spacepoints, downsampled_TrueEDep_spacepoints), f)
else:
    with open("downsampled_spacepoints.pkl", "rb") as f:
        downsampled_Tcluster_spacepoints, downsampled_Trec_spacepoints, downsampled_TrueEDep_spacepoints = pickle.load(f)


  0%|          | 0/70 [00:00<?, ?it/s]

# Categorize Spacepoints

In [6]:
len(downsampled_Tcluster_spacepoints)

70

In [7]:
real_nu_reco_nu_downsampled_spacepoints = []
real_nu_reco_cosmic_downsampled_spacepoints = []
real_cosmic_reco_nu_downsampled_spacepoints = []
real_cosmic_reco_cosmic_downsampled_spacepoints = []
far_from_vtx_downsampled_spacepoints = []

close_to_true_nu_spacepoint_threshold = 5
close_to_reco_nu_spacepoint_threshold = 5

for event_i in range(num_events):

    if len(downsampled_Tcluster_spacepoints[event_i]) == 0:
        real_nu_reco_cosmic_downsampled_spacepoints.append(np.empty((0, 3)))
        real_nu_reco_nu_downsampled_spacepoints.append(np.empty((0, 3)))
        real_cosmic_reco_nu_downsampled_spacepoints.append(np.empty((0, 3)))
        real_cosmic_reco_cosmic_downsampled_spacepoints.append(np.empty((0, 3)))
        far_from_vtx_downsampled_spacepoints.append(np.empty((0, 3)))
        continue

    # for T_cluster spacepoints, noting distances to true nu and reco nu spacepoints, and which are close to the reco nu vtx
    min_truth_dists = get_min_dists(downsampled_Tcluster_spacepoints[event_i][:, :3], downsampled_TrueEDep_spacepoints[event_i][:, :3])
    min_reco_nu_dists = get_min_dists(downsampled_Tcluster_spacepoints[event_i][:, :3], downsampled_Trec_spacepoints[event_i][:, :3])

    # assign features to spacepoints here
    close_to_truth_indices = np.where(min_truth_dists < close_to_true_nu_spacepoint_threshold)[0]
    far_from_truth_indices = np.where(min_truth_dists >= close_to_true_nu_spacepoint_threshold)[0]
    close_to_reco_nu_indices = np.where(min_reco_nu_dists < close_to_reco_nu_spacepoint_threshold)[0]
    far_from_reco_nu_indices = np.where(min_reco_nu_dists >= close_to_reco_nu_spacepoint_threshold)[0]

    # categorize spacepoints here
    real_nu_reco_nu_indices = np.intersect1d(close_to_reco_nu_indices, close_to_truth_indices)
    real_nu_reco_cosmic_indices = np.intersect1d(far_from_reco_nu_indices, close_to_truth_indices)
    real_cosmic_reco_nu_indices = np.intersect1d(close_to_reco_nu_indices, far_from_truth_indices)
    real_cosmic_reco_cosmic_indices = np.intersect1d(far_from_reco_nu_indices, far_from_truth_indices)

    real_nu_reco_cosmic_downsampled_spacepoints.append(downsampled_Tcluster_spacepoints[event_i][real_nu_reco_cosmic_indices, :])
    real_nu_reco_nu_downsampled_spacepoints.append(downsampled_Tcluster_spacepoints[event_i][real_nu_reco_nu_indices, :])
    real_cosmic_reco_nu_downsampled_spacepoints.append(downsampled_Tcluster_spacepoints[event_i][real_cosmic_reco_nu_indices, :])
    real_cosmic_reco_cosmic_downsampled_spacepoints.append(downsampled_Tcluster_spacepoints[event_i][real_cosmic_reco_cosmic_indices, :])


# Plot in 3D

In [8]:
include_non_downsampled_points = True

index = 61

run = rse["run"][index]
subrun = rse["subrun"][index]
event = rse["event"][index]

print("run, subrun, event:", run, subrun, event)


fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scene'}]])

# these are only added to set the camera at a better position
fig.add_trace(go.Scatter3d(
    x=expanded_detector_boundary_points[:, 2],
    y=expanded_detector_boundary_points[:, 0],
    z=expanded_detector_boundary_points[:, 1],
    mode='markers',
    marker=dict(
        size=0.2,
        color='black',
        opacity=0.8
    ),
    name='Expanded TPC Boundary'
))

fig.add_trace(go.Scatter3d(
    x=detector_boundary_points[:, 2],
    y=detector_boundary_points[:, 0],
    z=detector_boundary_points[:, 1],
    mode='markers',
    marker=dict(
        size=1,
        color='black',
        opacity=0.8
    ),
    name='TPC Boundary'
))

fig.add_trace(go.Scatter3d(
    x=[reco_nu_vtx[index][2]],
    y=[reco_nu_vtx[index][0]],
    z=[reco_nu_vtx[index][1]],
    mode='markers',
    marker=dict(size=10, color='purple', opacity=1),
    name='Reco Neutrino Vertex',
    visible='legendonly'
))

fig.add_trace(go.Scatter3d(
    x=[true_nu_vtx[index][2]],
    y=[true_nu_vtx[index][0]],
    z=[true_nu_vtx[index][1]],
    mode='markers',
    marker=dict(size=10, color='green', opacity=1),
    name='True Neutrino Vertex',
    visible='legendonly'

))


if include_non_downsampled_points:
    fig.add_trace(go.Scatter3d(
        x=Tcluster_spacepoints[index][:, 2],
        y=Tcluster_spacepoints[index][:, 0],
        z=Tcluster_spacepoints[index][:, 1],
        mode='markers',
        marker=dict(
            size=1,
            color="blue",
            opacity=0.8
        ),
        name='Tcluster Spacepoints',
        visible='legendonly'
    ))

fig.add_trace(go.Scatter3d(
    x=downsampled_Tcluster_spacepoints[index][:, 2],
    y=downsampled_Tcluster_spacepoints[index][:, 0],
    z=downsampled_Tcluster_spacepoints[index][:, 1],
    mode='markers',
    marker=dict(
        size=1,
        color="blue",
        opacity=0.8
    ),
    name='Downsampled Tcluster Spacepoints',
    visible='legendonly'
))

if include_non_downsampled_points:
    fig.add_trace(go.Scatter3d(
        x=Trec_spacepoints[index][:, 2],
        y=Trec_spacepoints[index][:, 0],
        z=Trec_spacepoints[index][:, 1],
        mode='markers',
        marker=dict(
            size=1,
            color='red',
            opacity=0.8
        ),
        name='Trec Spacepoints',
        visible='legendonly'
    ))

fig.add_trace(go.Scatter3d(
    x=downsampled_Trec_spacepoints[index][:, 2],
    y=downsampled_Trec_spacepoints[index][:, 0],
    z=downsampled_Trec_spacepoints[index][:, 1],
    mode='markers',
    marker=dict(
        size=1,
        color='red',
        opacity=0.8
    ),
    name='Downsampled Trec Spacepoints',
    visible='legendonly'
))

if include_non_downsampled_points:
    fig.add_trace(go.Scatter3d(
        x=TrueEDep_spacepoints[index][:, 2],
        y=TrueEDep_spacepoints[index][:, 0],
        z=TrueEDep_spacepoints[index][:, 1],
        mode='markers',
        marker=dict(
            size=1,
            color='orange',
            opacity=0.8
        ),
        name='TrueEDep Spacepoints',
        visible='legendonly'
    ))

fig.add_trace(go.Scatter3d(
    x=downsampled_TrueEDep_spacepoints[index][:, 2],
    y=downsampled_TrueEDep_spacepoints[index][:, 0],
    z=downsampled_TrueEDep_spacepoints[index][:, 1],
    mode='markers',
    marker=dict(
        size=3,
        color='orange',
        opacity=0.8
    ),
    name='Downsampled TrueEDep Spacepoints',
    visible='legendonly'
))

fig.add_trace(go.Scatter3d(
    x=real_nu_reco_nu_downsampled_spacepoints[index][:, 2],
    y=real_nu_reco_nu_downsampled_spacepoints[index][:, 0],
    z=real_nu_reco_nu_downsampled_spacepoints[index][:, 1],
    mode='markers',
    marker=dict(
        size=3,
        color='orange',
        opacity=0.8
    ),
    name='Real Nu Reco Nu Spacepoints',
))

fig.add_trace(go.Scatter3d(
    x=real_nu_reco_cosmic_downsampled_spacepoints[index][:, 2],
    y=real_nu_reco_cosmic_downsampled_spacepoints[index][:, 0],
    z=real_nu_reco_cosmic_downsampled_spacepoints[index][:, 1],
    mode='markers',
    marker=dict(
        size=3,
        color='red',
        opacity=0.8
    ),
    name='Real Nu Reco Cosmic Spacepoints',
))

fig.add_trace(go.Scatter3d(
    x=real_cosmic_reco_nu_downsampled_spacepoints[index][:, 2],
    y=real_cosmic_reco_nu_downsampled_spacepoints[index][:, 0],
    z=real_cosmic_reco_nu_downsampled_spacepoints[index][:, 1],
    mode='markers',
    marker=dict(
        size=3,
        color='brown',
        opacity=0.8
    ),
    name='Real Cosmic Reco Nu Spacepoints',
))

fig.add_trace(go.Scatter3d(
    x=real_cosmic_reco_cosmic_downsampled_spacepoints[index][:, 2],
    y=real_cosmic_reco_cosmic_downsampled_spacepoints[index][:, 0],
    z=real_cosmic_reco_cosmic_downsampled_spacepoints[index][:, 1],
    mode='markers',
    marker=dict(
        size=3,
        color='blue',
        opacity=0.8
    ),
    name='Real Cosmic Reco Cosmic Spacepoints',
))

fig.update_layout(
    scene=dict(
        xaxis_title='z',
        yaxis_title='x',
        zaxis_title='y',
        aspectratio=dict(
            x=5,
            y=3,
            z=1
        ),
    ),
    width=2000,
    height=1200,
    autosize=False,
    scene_camera=dict(
        eye=dict(x=-1.5, y=-1.5, z=1.5)
    )
)

fig.show(renderer="browser")


run, subrun, event: 20082 19 990
